# electronic-gadget-identifier
Machine learning project that identifies different electronic gadgets

* Jesse Båtman (TT2018-3A) - e1700826
* Joonatan Peltonen (TT2018-3B) - e1700807

In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms, models
from torch import nn, optim

In [ ]:
pretrained_size = 256
pretrained_means = [0.485, 0.456, 0.406]
pretrained_stds= [0.229, 0.224, 0.225]

transform = transforms.Compose([
                                transforms.Resize(pretrained_size),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean = pretrained_means, std = pretrained_stds)
                              ])

In [ ]:
trainset = datasets.ImageFolder(r'datasets\training', transform=transform)
validationset = datasets.ImageFolder(r'datasets\validation', transform=transform)

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
validationloader = torch.utils.data.DataLoader(validationset, batch_size=64, shuffle=True)

In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [ ]:
plt.imshow(images[1].permute(1, 2, 0).numpy().astype(np.uint8))

In [ ]:
figure = plt.figure()
num_of_images = 60
for index in range(1, num_of_images + 1):
    plt.subplot(6, 10, index)
    plt.axis('off')
    plt.imshow(images[index].permute(1, 2, 0).numpy().astype(np.uint8))

In [ ]:
import torchvision.models as models

pretrained_model = models.vgg16_bn(pretrained = True)

print(pretrained_model)

In [ ]:
classification_layer = nn.Sequential(nn.Linear(25088, 4096), 
                                     nn.ReLU(),
                                     nn.Dropout(p=0.3),
                                     nn.Linear(4096, 4),
                                     nn.LogSoftmax(dim=1))

pretrained_model.classifier =  classification_layer

print(pretrained_model.classifier)
model = pretrained_model

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
torch.set_num_threads(8)

In [ ]:
def run_conf_features(datasetloader):
    conv_features = []
    labels_list = []

    for images, labels in datasetloader:
        model.eval()
        with torch.no_grad():
            # modeling for each image batch
            conv_layer_output = model.features(images)        

            # flatten output as the next layers are the fully connected layers
            output = conv_layer_output.view(conv_layer_output.size(0), -1)

            conv_features.extend(output.data.cpu().numpy())
            labels_list.extend(labels.data.cpu().numpy())
            
    return (torch.Tensor(conv_features), torch.LongTensor(labels_list))

In [ ]:
conv_features, labels_list = run_conf_features(trainloader)

In [ ]:
conv_features_dataset = torch.utils.data.TensorDataset(conv_features, labels_list)
trainloader = torch.utils.data.DataLoader(conv_features_dataset, batch_size=64, shuffle=True)

In [ ]:
conv_features, labels_list = run_conf_features(validationloader)

In [ ]:
conv_features_dataset = torch.utils.data.TensorDataset(conv_features, labels_list)
validationloader = torch.utils.data.DataLoader(conv_features_dataset, batch_size=64, shuffle=True)

In [ ]:
time0 = time()
epochs = 3 # total number of iteration for training, training can be heavy, try first only one epoch.

# exercise 3: For plotting the model quality metrics.
epoch_list = []
training_loss_list = []
training_acc_list = []
validation_loss_list = []
validation_acc_list = []

for e in range(epochs):
    running_loss = 0
    # Variables to store the new quality metrics
    training_acc = 0
    validation_loss = 0
    validation_acc = 0
    for images, labels in trainloader:
        model.train()
        # CNN: flattening is not needed anymore, we expect 2d image.

        # defining gradient in each epoch as 0
        optimizer.zero_grad()
        
        # modeling for each image batch
        #conv_layer_output = model.features(images)        
        #conv_layer_output = model.avgpool(conv_layer_output)
        
        # remember to flatten before entering to the fully connected layer
        #conv_layer_output = conv_layer_output.view(conv_layer_output.size(0), -1)
        
        output = model.classifier(images)
        
        # calculating the loss
        loss = criterion(output, labels)
        
        # This is where the model learns by backpropagating
        loss.backward()
        
        # And optimizes its weights here
        optimizer.step()
        
        # calculating the loss
        running_loss += loss.item()
        
        
        # Exercise 3: calculate training accuracy
        with torch.no_grad():
            # Output of the network are log-probabilities, need to take exponential for probabilities
            _, predictions = torch.exp(output).max(1)
            # match predictions with true labels and count how many predictions are correct. Divide by batch size to get average for this batch.
            training_acc += torch.sum(predictions == labels).item() / len(images)            

    # Calculate validation loss and accuracy
    for images, labels in validationloader:
        with torch.no_grad():
            model.eval()
            # CNN: flattening is not needed anymore, we expect 2d image.

            # modeling for each image batch
            output = model.classifier(images)

            # calculating the loss
            loss = criterion(output, labels)

            # calculating the loss
            validation_loss += loss.item()

            # Output of the network are log-probabilities, need to take exponential for probabilities
            _, predictions = torch.exp(output).max(1)
            # match predictions with true labels and count how many predictions are correct. Divide by batch size to get average for this batch.
            validation_acc += torch.sum(predictions == labels).item() / len(images)            

    print("Epoch {} - Elapsed minutes {} - Train loss: {:0.5f}  Train acc: {:0.5f} Val loss: {:0.5f} Val acc: {:0.5f}".format(e, (time()-time0)/60,running_loss/len(trainloader),training_acc/len(trainloader), validation_loss/len(validationloader), validation_acc/len(validationloader)))
    epoch_list.append(e);
    training_loss_list.append(running_loss/len(trainloader))
    training_acc_list.append(training_acc/len(trainloader))
    validation_loss_list.append(validation_loss/len(validationloader))
    validation_acc_list.append(validation_acc/len(validationloader))

print("\nTraining Time (in minutes) =",(time()-time0)/60)

In [ ]:
plt.plot(epoch_list, training_loss_list, 'g', label='Training loss')
plt.plot(epoch_list, validation_loss_list, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(epoch_list, training_acc_list, 'g', label='Training accuracy')
plt.plot(epoch_list, validation_acc_list, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()